In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import copy
import glob
import os

In [2]:
class Miner:
    def __init__(self, num_spins):
        # Construct an instance with num_spins of desired spins.
        self.num_spins = num_spins
        self.hil_dim = 2**num_spins
        diag = np.empty(self.hil_dim)
        diag[:] = np.nan
        self.hamiltonian_diag = diag
    
    def random_clauses(self, num_clauses):
        # Sample num_clauses random clauses.
        self.num_clauses = num_clauses
        clauses = np.zeros((num_clauses, 3), dtype=int)
        for i in range(num_clauses):
            # Sample three spin indices.
            clauses[i] = np.random.choice(np.arange(self.num_spins), 3, replace=False)
        self.clauses = clauses
        
    def initial_state(self):
        # States contain 1, -1 spin orientations.
        self.state = (-1)**np.random.randint(0, 2, self.num_spins)
        return self.state
        
    def classic_energy(self, state):
        # Construct classical Hamiltonian from given state and clauses.
        energy = 0
        for i in range(self.num_clauses):
            energy += (state[self.clauses[i]].sum() - 1)**2
        return energy
    
    def flip_spin_proposal(self, ind_spin):
        # Propose energy and state of a single spin flip.
        neg_mat = np.ones(self.num_spins)
        neg_mat[ind_spin] *= -1
        proposed_state = neg_mat * self.state
        
        return self.classic_energy(proposed_state), proposed_state
        
    def mc_run(self, temperature):
        # Perform single MC run.
        rand_spin = np.random.randint(0, self.num_spins, 1)
        current_energy = self.classic_energy(self.state)
        proposal = self.flip_spin_proposal(rand_spin)
        delta_energy = proposal[0] - current_energy
        p = np.random.rand()
        if delta_energy < 0 or p < np.exp( - delta_energy / temperature):
            self.state[rand_spin] = - self.state[rand_spin]
        ind = self.hil_dim - 1 - self.binary_encode(proposal[1])
        self.hamiltonian_diag[ind] = proposal[0]
    
    def mc_loop(self, num_runs, temperature, callback):
        # Loop over MC runs.
        energies = np.zeros(num_runs)
        for i in range(num_runs):
            self.mc_run(temperature)
            if callback == 'energy':
                energies[i] = self.classic_energy(self.state)
        if callback == 'energy':
            return energies
        
    def binary_encode(self, state):
        # Representation of the state in the 2 basis.
        zero_one_rep = (1 - state[::-1])/2
        two_basis = 2**np.arange(self.num_spins)
        return np.sum(zero_one_rep * two_basis, dtype=int)
    
    # Annealer part
    
    def bin_decode(self, bin_rep):
        # Find spin representation from binary decoding.
        return (-1)**np.array(list(bin(bin_rep)[2:].zfill(self.num_spins)), dtype=int)
    
    def fill_hamiltonian(self):
        for i in range(self.hil_dim):
            self.hamiltonian_diag[i] = self.classic_energy(self.bin_decode(i))
        self.hamiltonian_diag = self.hamiltonian_diag[::-1]
            
    def field_term(self):
        sigma_0 = np.identity(2)
        sigma_x = np.array([[0, 1], [1, 0]])
        term = np.zeros((self.hil_dim, self.hil_dim))
        for i in range(self.num_spins):
            if i == 0:
                A = sigma_x
            else:
                A = sigma_0
            for j in range(1, self.num_spins):
                if i == j and i > 0:
                    A = np.tensordot(A, sigma_x, axes=0).swapaxes(1, 2)\
                        .reshape((2*A.shape[0], 2*A.shape[0]))
                else:
                    A = np.tensordot(A, sigma_0, axes=0).swapaxes(1, 2)\
                        .reshape((2*A.shape[0], 2*A.shape[0]))
            term += A
        return term
    
    def time_evolution(self, T, num_times, full_exp = True):
        I = np.identity(self.hil_dim)
        
        ts = np.linspace(0, T, num_times)
        delta_t = T / num_times

        fid_t = np.zeros((num_times,))
        
        H_p = np.diag(self.hamiltonian_diag).astype(complex)
        H_q = self.field_term().astype(complex)
        
        assert abs(H_p - H_p.T.conj()).max() < 1e-15
        assert abs(H_q - H_q.T.conj()).max() < 1e-15
        
        self.init_qstate = np.linalg.eigh(H_q)[1][:, 0]
        self.final_qstate = np.linalg.eigh(H_p)[1][:, 0]
        self.qstate = copy.copy(self.init_qstate)
        for i, t in enumerate(ts):
            h = 1 - t/T
            H = (1 - h) * H_p + h * H_q
            if full_exp:
                U = expm(- 1j * delta_t * H)
            else:
                U = I - 1j * delta_t * H - delta_t**2 / 2. * np.dot(H, H)
            self.qstate = np.dot(U, self.qstate)
            fid_t[i] = self.fidelity()
            
        return fid_t
            
    def fidelity(self):
        return abs(np.sum(self.final_qstate.T.conj()*self.qstate))
    
    def sample_ratio(self):
        return (self.hil_dim - np.isnan(self.hamiltonian_diag).sum()) / self.hil_dim

In [3]:
def instance(num_spins, num_clauses, num_runs_list, temperature_list, \
             anneal_T, anneal_steps):
    
    miner = Miner(num_spins)
    miner.random_clauses(num_clauses)
    
    # Monte Carlo part
    initial_state = miner.initial_state()
    res_mc_hamiltonian = np.zeros((len(num_runs_list), \
                                   len(temperature_list), \
                                   miner.hil_dim))
    for i, num_runs in enumerate(num_runs_list):
        for j, temperature in enumerate(temperature_list):
            miner.hamiltonian_diag[:] = np.nan
            miner.state = initial_state
            miner.mc_loop(num_runs, temperature, None)
            mc_hamiltonian = miner.hamiltonian_diag
            res_mc_hamiltonian[i, j] = mc_hamiltonian

            #print('runs {}, temperature {} -> sample ratio {}'\
            #      .format(num_runs, temperature, miner.sample_ratio()))
            
    # Annealing part
    miner.fill_hamiltonian()
    fid_time = miner.time_evolution(anneal_T, anneal_steps, full_exp = True)

    return miner.clauses, miner.hamiltonian_diag, res_mc_hamiltonian, fid_time


In [56]:
num_spins = 7
num_clauses_list = [4, 5, 6, 7, 8, 9]
num_instances = 1000
num_runs_list = [100, 1000, 10000]
temperature_list = [1, 2, 5, 10]
anneal_T = 10
anneal_steps = 500

In [ ]:
for num_clauses in num_clauses_list:
    
    res_clauses = np.zeros((num_instances, num_clauses, 3, ))
    res_hamiltonian_diag = np.zeros((num_instances, 2**num_spins, ))
    res_mc_samples = np.zeros((num_instances, len(num_runs_list), \
                               len(temperature_list), 2**num_spins, ))
    res_fidelity = np.zeros((num_instances, anneal_steps, ))
    
    for ind_instance in range(num_instances):
        print(f'num_clauses {num_clauses}, ind_instance {ind_instance}', end = '\r')
        
        clauses, hamiltonian_diag, mc_samples, fidelity = instance(num_spins, \
                                                                   num_clauses, \
                                                                   num_runs_list, \
                                                                   temperature_list, \
                                                                   anneal_T, \
                                                                   anneal_steps)

        res_clauses[ind_instance] = clauses
        res_hamiltonian_diag[ind_instance] = hamiltonian_diag
        res_mc_samples[ind_instance] = mc_samples
        res_fidelity[ind_instance] = fidelity
        
    number = 1
    while True:
        file_name = 'ec3_spins{}_clauses{}_instances{}_{}' \
                    .format(num_spins, num_clauses, num_instances, number)
        if not os.path.isfile(file_name + '.npz'):
            break
        else:
            number += 1

    np.savez(file_name + '.npz', \
             res_clauses = res_clauses, \
             res_hamiltonian_diag = res_hamiltonian_diag, \
             res_mc_samples = res_mc_samples, \
             res_fidelity = res_fidelity)
    
    with open(file_name + '.txt', 'w') as text_file:
        print(
            f'num_spins = {num_spins} \n' +
            f'num_clauses_list = {num_clauses_list} \n' +
            f'num_instances = {num_instances} \n' +
            f'num_runs_list = {num_runs_list} \n' +
            f'temperature_list = {temperature_list} \n' +
            f'anneal_T = {anneal_T} \n' +
            f'anneal_steps = {anneal_steps}', 
            file = text_file)